In [9]:
#https://github.com/lexfridman/mit-deep-learning/blob/master/tutorial_driving_scene_segmentation/tutorial_driving_scene_segmentation.ipynb

import tensorflow as tf

import os
from io import BytesIO
import tarfile
import tempfile
from six.moves import urllib

import matplotlib
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image
import cv2 as cv
from tqdm import tqdm
import IPython
from sklearn.metrics import confusion_matrix
from tabulate import tabulate

import warnings
warnings.simplefilter("ignore", DeprecationWarning)

In [8]:
class DeepLabModel(object):
    
    FROZEN_GRAPH_NAME = 'frozen_inference_graph'
    
    def __init__(self, tarball_path):
        self.graph = tf.Graph()
        graph_def = None
        
        tar_file = tarfile.open(tarball_path)
        
        for tar_info in tar_file.getmembers():
            if self.FROZEN_GRAPH_NAME in os.path.basename(tar_info.name):
                file_handle = tar_file.extractfile(tar_info)
                graph_def = tf.GraphDef.FromString(file_handle.read())
                break
        tar_file.close()
        
        
        if graph_def is None:
            raise RuntimeError('Cannot find inference graph in tar archive.')
            
        with self.graph.as_default():
            tf.import_graph_def(graph_def, name='')
        self.sess = tf.Session(graph=self.graph)
        
    def run(self, image, INPUT_TENSOR_NAME = 'ImageTensor:0', OUTPUT_TENSOR_NAME = 'SemanticPredistions:0'):
        
        width, height = image.size
        
        target_size = (2049, 1025)
        resized_image = image.convert('RGB').resize(target_size, Image.ANTIALIAS)
        batch_seg_map = self.sess.run(
            OUTPUT_TENSOR_NAME,
            feed_dict = {INPUT_TENSOR_NAME: [np.asarray(resized_image)]})
        seg_map = batch_seg_map[0]
        if len(seg_map.shape) == 2:
            seg_map = np.expand_dims(seg_map, -1)
        seg_map = cv.resize(seg_map, (width, height), interpolation=cv.INTER_NEAREST)
        return seg_map
        
        
        
            
            
            
        